In [1]:
import pandas as pd

df = pd.read_csv("/Users/kristinaspenner/Hiwi/SciEntsBank/data_SRAgraphs/SRA_allAnswers_promptST_25b2.tsv", sep="\t")

question = """Look at the picture of the bean seedling shortly after germination. Describe the function of each of the parts: cotyledon."""

df_correct = df.loc[df["Score"] == "1.0"]
df_contradict = df.loc[df["Score"] == "contradictory"]
df_part_corr_incompl = df.loc[df["Score"] == "partially_correct_incomplete"]
df_irrelevant = df.loc[df["Score"] == "irrelevant"]
df_non_domain = df.loc[df["Score"] == "non_domain"]

# Get the count of each score category
score_counts = df["Score"].value_counts()
print(score_counts)

# Extract counts for each category, defaulting to 0 if the category does not exist
count_correct = score_counts.get("1.0", 0)
count_contradictory = score_counts.get("contradictory", 0)
count_part_corr_incompl = score_counts.get("partially_correct_incomplete", 0)
count_irrelevant = score_counts.get("irrelevant", 0)
count_non_domain = score_counts.get("non_domain", 0)

df.head()

Score
partially_correct_incomplete    18
1.0                             14
irrelevant                       8
Name: count, dtype: int64


,AnswerId,PromptId,Score,AnswerText
0,ST.25b2.384.1,ST_25b2,partially_correct_incomplete,Helps give the plant food.
1,ST.25b2.385.1,ST_25b2,partially_correct_incomplete,Store food.
2,ST.25b2.390.1,ST_25b2,partially_correct_incomplete,Saves food.
3,ST.25b2.392.1,ST_25b2,1.0,Stores food for the seedling until it grows le...
4,ST.25b2.395.1,ST_25b2,1.0,Gives the seed food.


In [2]:
# Printing the items in the AnswerText column and defining the examples for the prompts

print("_____correct______")
print(df_correct["AnswerText"].to_list())
correct = df_correct["AnswerText"].to_list()

print("______pci______")
print(df_part_corr_incompl["AnswerText"].to_list())
part_corr_incompl = df_part_corr_incompl["AnswerText"].to_list()

print("______irrel______")
print(df_irrelevant["AnswerText"].to_list())
irrel = df_irrelevant["AnswerText"].to_list()

print("______contradict______")
print(df_contradict["AnswerText"].to_list())
contradict = df_contradict["AnswerText"].to_list()

print("______nondomain______")
print(df_non_domain["AnswerText"].to_list())
non_domain = df_non_domain["AnswerText"].to_list()

_____correct______
['Stores food for the seedling until it grows leaves.', 'Gives the seed food.', 'Feeds the seed.', 'Stores food for it until it grows into a plant.', 'To provide food when germination.', 'It gather up food for the seed.', 'Something that feeds the seed.', 'The cotyledon gives food to the plant until it grows roots.', 'It is the food supply until the plant grows leaves.', 'It stows the food for the young plant.', 'The cotyledon supplies food for the seed until it can supply its own food.', 'Stores food for the plant in until it can make on its own.', 'To give food to the baby plant.', 'It holds food for the plant before the roots grow.']
______pci______
['Helps give the plant food.', 'Store food.', 'Saves food.', 'To store the food, nutrients.', 'The cotyledon is the food storage.', 'Provide food.', 'Stores food.', 'The cotyledon stores the food.', 'The cotyledon is a little plant in the bean that store food.', 'Stores food.', 'Food supply.', 'A cotyledon in the food 

In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

print(llm.invoke("""In the next prompt I will give you a question with examples for different answers 
                to that question and I'd like you to generate more answers very similar to those in the example, ok?"""))


Absolutely! I'm ready when you are. Please provide the question and the examples, and I'll do my best to come up with more responses that are similar in style and tone to the ones provided.


In [4]:
# Prompt Templates for different answer categories 

prompt_template_corr = f""" 
                        Question: {{}}
                        Example Answers (Rating {{}}): 
                        {{}}
                        Generate {count_correct} more correct answers similar to the provided ones."""


prompt_template_contradict = f""" 
                        Question: {{}}
                        Example Answers (Rating {{}}): 
                        {{}}

                        Please generate {count_contradictory} more answers that are similar to the provided ones and belong to the category 'contradictory'.
    
                        'contradictory' means that answers are rather short and explicitly contradict the correct answer."""


prompt_template_pcinc = f""" 
                        Question: {{}}
                        Example Answers (Rating {{}}): 
                        {{}}

                        Please generate {count_part_corr_incompl} more answers that are similar to the provided ones and belong to the category 'partially correct 
                        or incomplete'.

                        'Partially correct or incomplete' means that it is a partially correct answer containing some but not all information 
                        from the correct answer."""


prompt_template_irrel = f""" 
                        Question: {{}}
                        Example Answers (Rating {{}}): 
                        {{}}

                        Generate {count_irrelevant} more answers similar to the provided ones that would be rated "irrelevant".
                        
                        'Irrelevant' in this context means 'the student answer is talking about domain content but not providing the 
                        necessary information'"""

# Function to remove everything in parentheses (if needed)
def remove(text):
    return re.sub(r'\(.*?\)', '', text)


In [5]:
rating = "correct"
prompt = prompt_template_corr.format(question, rating,
                                "\n".join(correct))

print(llm.invoke(prompt))


 Certainly! Here are 14 additional correct answers for the function of the cotyledon in a bean seedling:

1. Provides nutrients for the developing plant until it can produce its own food through photosynthesis.
2. Acts as an energy store for the young seedling, allowing it to grow and develop without expending all of its energy on germination.
3. Serves as a temporary food source for the seedling during its early stages of growth.
4. Helps regulate the water content of the seedling, ensuring that it stays hydrated during the critical period after germination.
5. Provides a safe and stable environment for the developing plant, protecting it from external factors that could harm it.
6. Allows the seedling to focus its energy on growing roots and shoots, rather than expending energy on finding food and water.
7. Enables the seedling to grow and develop more quickly and efficiently, thanks to the stored nutrients in the cotyledon.
8. Acts as a "life raft" for the seedling, providing suste

In [11]:
import re

corr = """
1. Provides nutrients for the developing plant until it can produce its own food through photosynthesis.
2. Acts as an energy store for the young seedling, allowing it to grow and develop without expending all of its energy on germination.
3. Serves as a temporary food source for the seedling during its early stages of growth.
4. Helps regulate the water content of the seedling, ensuring that it stays hydrated during the critical period after germination.
5. Provides a safe and stable environment for the developing plant, protecting it from external factors that could harm it.
6. Allows the seedling to focus its energy on growing roots and shoots, rather than expending energy on finding food and water.
7. Enables the seedling to grow and develop more quickly and efficiently, thanks to the stored nutrients in the cotyledon.
8. Acts as a "life raft" for the seedling, providing sustenance during the first critical stages of growth when it is vulnerable and unable to produce its own food.
9. Helps establish a strong root system, which is essential for the plant's long-term survival and success.
10. Allows the seedling to allocate more energy to growing roots and shoots, leading to faster establishment and growth.
11. Provides a source of nutrients that the developing plant can use to fuel its growth and development.
12. Helps regulate the temperature of the seedling during the critical period after germination, ensuring that it stays at a stable temperature for optimal growth.
13. Acts as an energy reserve for the seedling, allowing it to withstand stressors such as drought or extreme temperatures.
14. Plays a critical role in the plant's early life stages, providing the foundation for its future growth and success.
"""

# Use regular expressions to remove the leading number and period from each sentence
corr_answers = re.sub(r'^\d+\.\s', '', corr, flags=re.MULTILINE)

# Split the cleaned text into individual sentences and store in a list
corr_answers = corr_answers.strip().split('\n')

# Print the cleaned result
print(corr_answers)
len(corr_answers)

['Provides nutrients for the developing plant until it can produce its own food through photosynthesis.', 'Acts as an energy store for the young seedling, allowing it to grow and develop without expending all of its energy on germination.', 'Serves as a temporary food source for the seedling during its early stages of growth.', 'Helps regulate the water content of the seedling, ensuring that it stays hydrated during the critical period after germination.', 'Provides a safe and stable environment for the developing plant, protecting it from external factors that could harm it.', 'Allows the seedling to focus its energy on growing roots and shoots, rather than expending energy on finding food and water.', 'Enables the seedling to grow and develop more quickly and efficiently, thanks to the stored nutrients in the cotyledon.', 'Acts as a "life raft" for the seedling, providing sustenance during the first critical stages of growth when it is vulnerable and unable to produce its own food.',

14

In [6]:
print(llm.invoke("""In the next prompt I will give you the same question with examples for answers that 
                were rated 'partially correct or incomplete' and I'd like you to generate more answers very similar to those in the example, 
                ok?"""))


Of course! I'd be happy to help. Please provide the question with the partially correct or incomplete answers, and I'll do my best to generate additional responses that are similar in style and content.


In [7]:
rating = "partially correct or incomplete"
prompt = prompt_template_pcinc.format(question, rating,
                                "\n".join(part_corr_incompl))

print(llm.invoke(prompt))


Of course! Here are 18 more answers that are partially correct or incomplete for the question "Describe the function of each of the parts: cotyledon":

1. Helps give the plant water. (Incomplete)
2. Provides nutrients to the growing plant. (Partially correct)
3. Acts as a protective covering for the developing seedling. (Incomplete)
4. Stores energy for the plant's growth and development. (Partially correct)
5. Allows the plant to absorb oxygen from the surrounding environment. (Incorrect)
6. Holds onto water and nutrients until they are needed by the plant. (Incomplete)
7. Transports nutrients and water throughout the plant. (Incorrect)
8. Serves as a shelter for the developing seedling. (Incomplete)
9. Provides a source of food for the growing plant. (Partially correct)
10. Acts as a barrier to protect the plant from harmful substances in the soil. (Incomplete)
11. Stores nutrients and water for the plant's growth and development. (Partially correct)
12. Supports the growth of the p

In [12]:
answers = """
1. Helps give the plant water. (Incomplete)
2. Provides nutrients to the growing plant. (Partially correct)
3. Acts as a protective covering for the developing seedling. (Incomplete)
4. Stores energy for the plant's growth and development. (Partially correct)
5. Allows the plant to absorb oxygen from the surrounding environment. (Incorrect)
6. Holds onto water and nutrients until they are needed by the plant. (Incomplete)
7. Transports nutrients and water throughout the plant. (Incorrect)
8. Serves as a shelter for the developing seedling. (Incomplete)
9. Provides a source of food for the growing plant. (Partially correct)
10. Acts as a barrier to protect the plant from harmful substances in the soil. (Incomplete)
11. Stores nutrients and water for the plant's growth and development. (Partially correct)
12. Supports the growth of the plant by providing necessary nutrients. (Incorrect)
13. Helps regulate the plant's water and nutrient intake. (Incomplete)
14. Protects the developing seedling from excessive light or heat. (Incomplete)
15. Provides a habitat for beneficial microorganisms that help the plant grow. (Incorrect)
16. Holds onto nutrients and water until they are needed by the plant. (Incomplete)
17. Regulates the plant's growth and development through hormone production. (Incorrect)
18. Acts as a conduit for water and nutrient transport between the roots and shoots of the plant. (Incomplete)
"""

answers = remove(answers)

pcincl_answers = re.sub(r'^\d+\.\s', '', answers, flags=re.MULTILINE)

# Split the cleaned text into individual sentences and store in a list
pcincl_answers = pcincl_answers.strip().split('\n')

print(pcincl_answers)
len(pcincl_answers)

['Helps give the plant water. ', 'Provides nutrients to the growing plant. ', 'Acts as a protective covering for the developing seedling. ', "Stores energy for the plant's growth and development. ", 'Allows the plant to absorb oxygen from the surrounding environment. ', 'Holds onto water and nutrients until they are needed by the plant. ', 'Transports nutrients and water throughout the plant. ', 'Serves as a shelter for the developing seedling. ', 'Provides a source of food for the growing plant. ', 'Acts as a barrier to protect the plant from harmful substances in the soil. ', "Stores nutrients and water for the plant's growth and development. ", 'Supports the growth of the plant by providing necessary nutrients. ', "Helps regulate the plant's water and nutrient intake. ", 'Protects the developing seedling from excessive light or heat. ', 'Provides a habitat for beneficial microorganisms that help the plant grow. ', 'Holds onto nutrients and water until they are needed by the plant. '

18

In [ ]:
print(llm.invoke("""In the next prompt I will give you the same question with examples for answers that 
                were rated 'contradictory' and I'd like you to generate more answers very similar to those in the example, 
                ok?"""))

In [ ]:
rating = "contradictory"
prompt = prompt_template_contradict.format(question, rating,
                                "\n".join(contradict))

print(llm.invoke(prompt))

In [ ]:
answers = """
"""

contradict_answers = re.sub(r'^\d+\.\s', '', answers, flags=re.MULTILINE)

# Split the cleaned text into individual sentences and store in a list
contradict_answers = contradict_answers.strip().split('\n')

print(contradict_answers)
len(contradict_answers)

In [9]:
print(llm.invoke("""In the next prompt I will give you the same question with examples for answers that 
                were rated 'irrelevant' and I'd like you to generate more answers very similar to those in the example, 
                ok?"""))


Of course! I'd be happy to help. Please provide the question with the irrelevant answers, and I'll do my best to come up with more similar responses.


In [10]:
rating = "irrelevant"
prompt = prompt_template_irrel.format(question, rating,
                                "\n".join(irrel))

print(llm.invoke(prompt))


Of course! Here are 8 more irrelevant answers to the question about the function of the cotyledon:

1. The cotyledon is where the seed gets its superpowers.
2. The cotyledon helps the seed grow by giving it a hug.
3. The cotyledon is the magic trick that makes the seed grow bigger.
4. The cotyledon is like a secret hideout for the seed to rest in.
5. The cotyledon holds the seed's breath until it's ready to grow.
6. The cotyledon is the seed's built-in GPS, helping it find its way to the sunlight.
7. The cotyledon is where the seed stores all its nutrients for growth.
8. The cotyledon is like a special suit that helps the seed blend in with its surroundings.


In [13]:
answers = """
1. The cotyledon is where the seed gets its superpowers.
2. The cotyledon helps the seed grow by giving it a hug.
3. The cotyledon is the magic trick that makes the seed grow bigger.
4. The cotyledon is like a secret hideout for the seed to rest in.
5. The cotyledon holds the seed's breath until it's ready to grow.
6. The cotyledon is the seed's built-in GPS, helping it find its way to the sunlight.
7. The cotyledon is where the seed stores all its nutrients for growth.
8. The cotyledon is like a special suit that helps the seed blend in with its surroundings.
"""

#answers = remove(answers)

irrel_answers = re.sub(r'^\d+\.\s', '', answers, flags=re.MULTILINE)

# Split the cleaned text into individual sentences and store in a list
irrel_answers = irrel_answers.strip().split('\n')

print(irrel_answers)
len(irrel_answers)

['The cotyledon is where the seed gets its superpowers.', 'The cotyledon helps the seed grow by giving it a hug.', 'The cotyledon is the magic trick that makes the seed grow bigger.', 'The cotyledon is like a secret hideout for the seed to rest in.', "The cotyledon holds the seed's breath until it's ready to grow.", "The cotyledon is the seed's built-in GPS, helping it find its way to the sunlight.", 'The cotyledon is where the seed stores all its nutrients for growth.', 'The cotyledon is like a special suit that helps the seed blend in with its surroundings.']


8

In [17]:
non_domain_answers = ["I do not know", "I do not know"]

In [14]:
import pandas as pd

PromptId = question
Score = "1.0" 
AnswerText = corr_answers

df1 = pd.DataFrame({"PromptId": PromptId,
                   "Score" : Score,
                   "AnswerText" : AnswerText})

df1.index = df1.index+1

PromptId = question
Score = "irrelevant" 
AnswerText = irrel_answers

df2 = pd.DataFrame({"PromptId": PromptId,
                   "Score" : Score,
                   "AnswerText" : AnswerText})

df2.index = df2.index+1

PromptId = question
Score = "partially_correct_incomplete" 
AnswerText = pcincl_answers

df3 = pd.DataFrame({"PromptId": PromptId,
                   "Score" : Score,
                   "AnswerText" : AnswerText})

df3.index = df3.index+1

df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
df.index = df.index+1
df.head(50)

,PromptId,Score,AnswerText
1,Look at the picture of the bean seedling short...,1.0,Provides nutrients for the developing plant un...
2,Look at the picture of the bean seedling short...,1.0,Acts as an energy store for the young seedling...
3,Look at the picture of the bean seedling short...,1.0,Serves as a temporary food source for the seed...
4,Look at the picture of the bean seedling short...,1.0,Helps regulate the water content of the seedli...
5,Look at the picture of the bean seedling short...,1.0,Provides a safe and stable environment for the...
6,Look at the picture of the bean seedling short...,1.0,Allows the seedling to focus its energy on gro...
7,Look at the picture of the bean seedling short...,1.0,Enables the seedling to grow and develop more ...
8,Look at the picture of the bean seedling short...,1.0,"Acts as a ""life raft"" for the seedling, provid..."
9,Look at the picture of the bean seedling short...,1.0,"Helps establish a strong root system, which is..."
10,Look at the picture of the bean seedling short...,1.0,Allows the seedling to allocate more energy to...


In [15]:
# shuffle the DataFrame rows
df = df.sample(frac = 1)
 
# print the shuffled DataFrame
print("\nShuffled DataFrame:")
print(df)

df.to_csv("SRA_allAnswers_promptST_25b2.csv", index=False)


Shuffled DataFrame:
                                             PromptId  \
9   Look at the picture of the bean seedling short...   
14  Look at the picture of the bean seedling short...   
27  Look at the picture of the bean seedling short...   
33  Look at the picture of the bean seedling short...   
38  Look at the picture of the bean seedling short...   
31  Look at the picture of the bean seedling short...   
26  Look at the picture of the bean seedling short...   
20  Look at the picture of the bean seedling short...   
8   Look at the picture of the bean seedling short...   
3   Look at the picture of the bean seedling short...   
5   Look at the picture of the bean seedling short...   
10  Look at the picture of the bean seedling short...   
25  Look at the picture of the bean seedling short...   
19  Look at the picture of the bean seedling short...   
13  Look at the picture of the bean seedling short...   
24  Look at the picture of the bean seedling short...   
11  Look a